Quete 3 scrapping Allo Cine

### Challenge

-Essaye de calculer l'âge moyen des acteurs et actrices dans le film Interstellar.

### age moyen des acteurs du film interstellar

In [9]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup


#  fonction recensant les liens du site allocine d'un producteur de films
def dataframe_liens_producteur(url):
    try:
        # Contournement d'éventuelles protections anti-scrapping
        navigateur = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

        # Faire une requête HTTP pour obtenir le contenu de la page
        reponse = requests.get(url, headers={"User-Agent": navigateur})

        # Parser le contenu HTML avec BeautifulSoup
        soupe = BeautifulSoup(reponse.text, "html.parser")
        # on ne prend que le body
        soupe = soupe.body
        # on supprime les balises scripts
        for script in soupe("script"):
            script.decompose()

        # Trouver tous les liens sur la page
        liens = soupe.find_all("a")

        # Filtrer uniquement sur les films dont l'artiste est le réalisateur
        soupe_directeur = soupe.find("div", {"class": "gd-col-left"})

        # Chercher tous les liens contenus dans cet encadré
        liens = soupe_directeur.find_all("a")

        # Créer un dataframe avec les titres et les liens
        df_url_producteur = pd.DataFrame(
            {"title": [i["title"] for i in liens], "href": [i["href"] for i in liens]}
        )

        return df_url_producteur
    except Exception as e:
        print(f"Erreur dans dataframe_liens_producteur: {e}")
        return None


# ----------------------------------------------------------------
# Récupère les liens des acteurs (casting) à partir de l'URL du film
def obtenir_liens_acteurs(url_court):
    try:
        url = "https://www.allocine.fr" + url_court
        html = requests.get(url)
        soupe = BeautifulSoup(html.text, "html.parser")
        # on ne prend que le body
        soupe = soupe.body
        # on supprime les balises scripts
        for script in soupe("script"):
            script.decompose()
        # liste de tous les éléments a avec la classe “meta-title-link” qui sont à l’intérieur d’un élément div avec la classe “card person-card person-card-col”
        sections_acteurs=soupe.select("div.card.person-card.person-card-col a.meta-title-link")
        sections_acteurs += soupe.select("div.gd.gd-xs-1.gd-s-2.md-table-row a")
        # print(f"\nsections_acteurs :\n{sections_acteurs} \n")
        liens_acteurs = []
        for acteur in sections_acteurs:
            if acteur is not None:
                liens_acteurs.append(acteur["href"])
        # print(f"\nliens_acteurs :\n{len(liens_acteurs)} \n")
        # print(f"\nliens_acteurs :\n{liens_acteurs} \n")
        return liens_acteurs
    except Exception as e:
        print(f"Erreur dans obtenir_liens_acteurs: {e}")
        return []

# ----------------------------------------------------------------
# Récupère l'âge de l'acteur à partir de son lien
def obtenir_age_acteur(lien_acteur):
    try:
        url = "https://www.allocine.fr" + lien_acteur
        # print(f"\nurl :\n{url} \n")
        html = requests.get(url)
        soupe = BeautifulSoup(html.text, "html.parser")
        # on ne prend que le body
        soupe = soupe.body
        # on supprime les balises scripts
        for script in soupe("script"):
            script.decompose()
        # elements_meta = soupe.find_all("div", {"class": "meta-body-item"})
        # Regex "Age\s*" correspond à "Age" suivi de zéro ou plusieurs espaces
        element_age = soupe.find("span", string=re.compile(r"Age\s*", re.IGNORECASE))
        if element_age:
            # Trouve le prochain élément frère qui est une balise <strong>
            age = element_age.find_next_sibling("strong")
            if age:
                # print(f"\nage.text :\n{age.text} \n")
                return age.text
        return None
    except Exception as e:
        print(f"Erreur dans obtenir_age_acteur: {e}")
        return None


# ----------------------------------------------------------------
# Calcule l'âge moyen des acteurs à partir de leurs liens
def calculer_age_moyen(liens_acteurs):
    try:
        ages = []
        for lien in liens_acteurs:
            age = obtenir_age_acteur(lien)
            # Vérifie si l'âge est un nombre
            if age is not None and age.isdigit():
                ages.append(int(age))
        print(f"\nages :\n{ages} \n")
        age_moyen = sum(ages) / len(ages) if ages else None
        age_moyen = round(age_moyen, 2)
        return age_moyen
    except Exception as e:
        print(f"Erreur dans calculer_age_moyen: {e}")
        return None


# ----------------------------------------------------------------
# Fonction principale pour obtenir l'âge moyen des acteurs d'un film d'un producteur au choix
def age_moyen_acteur(nom_film, my_dataframe):
    try:
        nom_film = nom_film.capitalize()
        # print(f"\nnom_film :\n{nom_film} \n")
        # Construit l'URL du film à partir du dataframe
        url_film = (
            my_dataframe["href"][my_dataframe["title"] == nom_film]
            .values.item()
            .replace(".html", "")
            .replace("_gen_cfilm=", "-")
        )
        url_casting = (
            url_film.replace(".html", "").replace("_gen_cfilm=", "-") + "/casting/"
        )
        # print(f"\nurl_film :\n{url_film} \n")
        # print(f"\nurl_casting :\n{url_casting} \n")
        liens_acteurs = obtenir_liens_acteurs(url_casting)
        age_moyen = calculer_age_moyen(liens_acteurs)
        # print(f"\nliens_acteurs :\n{liens_acteurs} \n")
        # print(f"\nage_moyen :\n{age_moyen} \n")
        if age_moyen is not None:
            print(f"L'âge moyen des acteurs du film {nom_film} est de {age_moyen} ans.")
            print(f"Au moment du tournage du film ils avaient {age_moyen-9} ans.")
        else:
            print("Aucun âge trouvé pour les acteurs du film.")
    except Exception as e:
        print(f"Erreur dans age_moyen_acteur: {e}")

In [61]:
# Test fonction age moyen
# url du producteur Christopher Nolan
url = "https://www.allocine.fr/personne/fichepersonne-30367/filmographie/"
nom_film = "interstellar"
my_dataframe1 = dataframe_liens_producteur(url)
# my_dataframe = df_url

age_moyen_acteur(nom_film, my_dataframe1)


ages :
[53, 54, 41, 90, 78, 46, 48, 23, 45, 44, 28, 45, 47, 49, 86, 91, 53, 40, 48] 

L'âge moyen des acteurs du film Interstellar est de 53.11 ans.
Au moment du tournage du film ils avaient 44.11 ans.
